# Namespaces `⎕NS`

> The utility of a language as a tool of thought increases with the range of topics it can treat, but decreases with the amount of vocabulary and the complexity of grammatical rules which the user must keep in mind. Economy of notation is therefore important. --_Kenneth E. Iverson_

A [namespace](http://help.dyalog.com/18.0/index.htm#Language/Introduction/Namespaces/Namespaces.htm) is a way to group data and code into a hierarchy. Dyalog describes namespaces like so:

> Namespace is a (class 9) object in Dyalog APL. Namespaces are analogous to nested workspaces.

No, that doesn't mean anything to me either. In fact, APL namespaces are similar in spirit to those found in [C++](https://en.cppreference.com/w/cpp/language/namespace):

> Namespaces provide a method for preventing name conflicts in large projects. Symbols declared inside a namespace block are placed in a named scope that prevents them from being mistaken for identically-named symbols in other scopes. 

Here's an anonymous namespace:

In [1]:
obj ← ⎕NS⍬

done

Rebuilding user command cache... done

We can assign values to variables inside this namespace:

In [2]:
obj.(name cost id)  ← 'widget' 55.0 'widg443'

In [3]:
obj.(name cost id)

┌──────┬──┬───────┐
│widget│55│widg443│
└──────┴──┴───────┘

Names inside a namespace can hold any value that names can hold _outside_ a namespace, including functions:

In [4]:
obj.sum←+/

In [5]:
obj.sum 1 2 3 4 5

15

## Scripted namespaces

A nify feature is that we can compose a namespace as a _script_. In RIDE, you need to say `)ed ⍟ mynamespace` in order to work with a scripted namespace. Yes, of course that's a good use for the _logarithm_ glyph. It looks like so:

![ns1](./IMG/ns1.png)

In this way, the namespace can be a convenient way to organise your code. And in case it wasn't obvious, it's actually a way in which you can have many multi-line dfns in the same editing window -- or even text file. It even works as intended in the Jupyter notebook:

In [6]:
]dinput
:Namespace myns
    f ← {
        ⍺+⍵
    }

    g ← {
        ⍝ g fun
        ⍺⍵
    }

    h ← {
        s ← '\d+'⎕R'D'⊢⍵
        ⍺ g s
    }
:EndNamespace

In [7]:
'nodigits' myns.h 'abd556jashgd8879'

┌────────┬───────────┐
│nodigits│abdDjashgdD│
└────────┴───────────┘

## Wait, this is starting to look like a dict!

We can get tantalisingly close to having a namespace function as a dict. In order to list the names of variables contained in a namespace, we have the [namelist](http://help.dyalog.com/18.0/index.htm#Language/System%20Functions/nl.htm), `⎕NL`, system function.

In [8]:
⊢names ← obj.⎕NL 2

cost
id 
name

The `2` there lets `⎕NL` know that we want an array back.

In [9]:
⍴names

3 4

If we feed it `¯2` instead we get a nested vector instead:

In [10]:
obj.⎕NL ¯2

┌────┬──┬────┐
│cost│id│name│
└────┴──┴────┘

The right argument to `⎕NL` is a [name class](http://help.dyalog.com/18.0/index.htm#Language/System%20Functions/nc.htm), allowing us to select based on what kind something contained in the namespace is. Many of the name classes are concerned with bits of Dyalog that are out of scope for this book.

We can get the _values_ of variables by evaluating their names:

In [11]:
obj⍎'cost'
obj.(⍎¨⎕NL ¯2)

55

┌──┬───────┬──────┐
│55│widg443│widget│
└──┴───────┴──────┘

We can get tantalisingly close to having a namespace function as a dict:

In [12]:
_set←{⍎'⍺⍺.',⍺,'←⍵'⊣⍺⍺}
keyz←{⍵.⎕NL ¯2}
vals←{⍵.(⍎¨⎕NL ¯2)}

In [13]:
'hello' (obj _set) 'world'

In [14]:
obj⍎'hello'
keyz obj
vals obj

world

┌────┬─────┬──┬────┐
│cost│hello│id│name│
└────┴─────┴──┴────┘

┌──┬─────┬───────┬──────┐
│55│world│widg443│widget│
└──┴─────┴───────┴──────┘

but this approach won't let you use anything but character vectors as keys. There are also some performance constraints if the number of items in a namespace grow large. 

## A note on mutability

Namespaces in Dyalog are reference types, and mutable. This allows you to bypass some scope-related barriers that may have been erected for very good reasons, so _caveat emptor_. For example, we can mutate a namespace even if it's passed as the _left_ argument, which is an error for normal arrays:

In [15]:
ns ← ⎕NS⍬
ns.key ← 45
ns {⍺.key ← 99 ⋄ ⍵} 'hello' ⍝ Mutation through ⍺...
ns.key

hello

99

It also means that there is no need for modified assignment through a tack, `⊢←` if we want to set a value in a namespace not in our immediate scope. All of this is either very useful, or very dangerous, depending on your particular view point. The reality is that it's both useful, but also increases the risk of fot-gun incidents.